In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

# Preprocess Time of Major and Elect

In [4]:
total_regis = pd.read_csv('./Data/Preprocess/total_regis_2.csv')

In [5]:
def get_day_and_hour(x):
    (day1, day2) = (x[0], x[1])
    (hour1, hour2) = (x[2], x[3])

    if day2 == 0:
        time1 = day1 * 10 + hour1
        time2 = day1 * 10 + hour2

    else:
        time1 = day1 * 10 + hour1
        time2 = day2 * 10 + hour1
    
    return [time1, time2]

In [6]:
total_regis['시간'] = total_regis[['요일1', '요일2', '시간1', '시간2']].apply(get_day_and_hour, axis = 1)

# Define Functions

In [8]:
def get_add_time_impos(x):
    time_list = []

    time = x.split()

    day_dict = {'월': 1, '화': 2, '수': 3, '목': 4, '금': 5, '토': 6, '일': 7}
    
    day = day_dict[time[0]]

    if len(time) == 1:
        for i in range(1, 10):
            y = day * 10 + i
            time_list.append(y)

    else:
        y = day * 10 + int(time[1])
        time_list.append(y)

    return time_list

In [9]:
def get_suitable(x, year):
    value = 1

    if abs(x - year) > 1:
        value = 0

    return value

In [10]:
def get_register(x, register_list):
    value = 0

    if x in register_list:
        value = 1

    return value

In [11]:
def get_place_impos(time, place, place_dict):
    for t in time:
        
        if t in place_dict[place]:
            return 1
            
    return 0

In [12]:
def preprocess_place(data, place_data):
    place_impos_dict = {0: [], 1: [], 2: []}

    for idx in range(len(place_data)):
        time = place_data['시간'].iloc[idx]
        place = place_data['장소'].iloc[idx]

        if place == 1:
            place_impos = 2

        else:
            place_impos = 1

        for t in time:
            place_impos_dict[place_impos] += [t - 1, t + 1]

    data['거리'] = data[['시간', '장소']].apply(lambda x: get_place_impos(x[0], x[1], place_impos_dict), axis = 1)

    return data

In [13]:
def get_time_impos(time, time_dict):
    for t in time:
        
        if time_dict[t] == 1:
            return 1

    return 0

In [14]:
def get_time_succe(time, time_dict):
    for t in time:

        if time_dict[t] == 1:
            return 1

    return 0

In [15]:
def preprocess_time(data, time_data, add_time_list):

    time_impos_list = time_data['시간'].sum() + add_time_list

    time_impos_dict = {0: 0}
    time_succe_dict = {0: 0}

    (time_previ, time_impos, time_succe) = (-1, -1, -1)

    for i in range(1, 8):
        time_last = i * 10

        for j in range(0, 10):
            time = i * 10 + j

            time_succe_dict[time] = time_succe

            if time in time_impos_list:
                time_impos = 1

            else:
                time_impos = 0
                time_last = time

            time_impos_dict[time] = time_impos

            if (time_impos, time_previ) == (1, 1):
                time_succe = 1
                time_succe_dict[time_last] = time_succe

            else:
                time_succe = 0

            time_previ = time_impos

    time_succe_dict[10] = 0

    data['중복'] = data['시간'].apply(lambda x: get_time_impos(x, time_impos_dict))
    data['연강'] = data['시간'].apply(lambda x: get_time_succe(x, time_succe_dict))

    return data

In [16]:
def filter_data(data, cols, cols_dict):
    for col in cols:
        data = data.loc[data[col] == cols_dict[col]]

    return data

# Run Program

In [18]:
total_regis_custom = total_regis.copy()

In [19]:
year = int(input('학년 :'))

regis_now_list = []

while True:
    
    number = input('신청 과목 : ')

    if number == '':
        break

    regis_now_list.append(number)

add_time_list = []

while True:

    time = input('제외 시간 :')

    if time == '':
        break

    add_time_list += get_add_time_impos(time)

print('입력 종료')

학년 : 4
신청 과목 :  AMTH3003-00
신청 과목 :  AMTH3013-00
신청 과목 :  AMTH3027-00
신청 과목 :  
제외 시간 : 금
제외 시간 : 


입력 종료


In [20]:
total_regis_custom['권장'] = total_regis_custom['대상학년'].apply(lambda x: get_suitable(x, year))

In [ ]:
while True:

    total_regis_custom['신청'] = total_regis_custom['학수번호-분반'].apply(lambda x: get_register(x, regis_now_list))
    
    regis_now = total_regis_custom.loc[((total_regis_custom['신청'] == 1) & (total_regis_custom['온라인'] == 0))] 

    total_regis_custom = preprocess_place(total_regis_custom, regis_now)
    total_regis_custom = preprocess_time(total_regis_custom, regis_now, add_time_list)

    cols = ['학수번호-분반', '강좌명', '대상학년', '교수명', '학점', '특이사항']

    cols_filter = ['수강', '중복']
    
    cols_filter_dict = {
        '전공1': 1, '전공2': 1, '교양': 1, '필수': 1, '온라인': 1, '수강': 0, '권장': 1,
        '신청': 0, '거리': 0, '중복': 0, '연강': 0
    }
    
    print()

    while True:

        total_regis_custom['신청'] = total_regis_custom['학수번호-분반'].apply(lambda x: get_register(x, regis_now_list))

        regis_now = total_regis_custom.loc[((total_regis_custom['신청'] == 1) & (total_regis_custom['온라인'] == 0))]
        
        display(regis_now[cols])

        print()
        number = input('취소 (학수번호-분반) :')

        if number == '':
            break

        regis_now_list.remove(number)
    
    print()
    print('옵션 : 전공1 전공2 교양 필수 온라인 권장 거리 연강')

    while True:
        col = input('필터 선택 :')

        if col == '':
            break

        cols_filter.append(col)

    total_regis_filter = filter_data(total_regis_custom, cols_filter, cols_filter_dict)

    cols = ['학수번호-분반', '강좌명', '대상학년', '교수명', '학점', '특이사항']

    idx = 0

    print()
    print('다음 항목 탐색 시 n 입력')

    while True:
        print()
        display(total_regis_filter[cols].iloc[idx: idx + 5])

        number = input('신청 (학수번호-분반) :')

        if number == '':
            break

        elif number == 'n':
            idx += 5

        else:
            regis_now_list.append(number)
            total_regis_filter = total_regis_filter.loc[total_regis_filter['학수번호-분반'] != number]

,학수번호-분반,강좌명,대상학년,교수명,학점,특이사항
0,AMTH3003-00,미분기하학,3,한창용,3,NaN
2,AMTH3013-00,현대대수학I,3,김정산,3,NaN
7,AMTH3027-00,수치선형대수,3,김경수,3,NaN


취소 (학수번호-분반) : 



옵션 : 전공1 전공2 교양 필수 온라인 권장 거리 연강


필터 선택 : 전공2
필터 선택 : 권장
필터 선택 : 



다음 항목 탐색 시 n 입력



...

,학수번호-분반,강좌명,대상학년,교수명,학점,특이사항
3,AMTH3023-00,독립심화학습1(응용수학과),3,박준표,3,NaN
9,AMTH3022-00,응용수학 캡스톤디자인 2,3,김경수,3,강의일시 변동 가능
11,AMTH4018-00,최적화이론,4,김정호,3,NaN
13,AMTH3028-00,편미분방정식,4,김정호,3,NaN
267,AMTH4012-00,졸업논문(응용수학과),4,김경수,0,NaN


신청 (학수번호-분반) : 


,학수번호-분반,강좌명,대상학년,교수명,학점,특이사항
0,AMTH3003-00,미분기하학,3,한창용,3,NaN
2,AMTH3013-00,현대대수학I,3,김정산,3,NaN
4,AMTH3029-00,복소함수및응용,3,박준표,3,NaN
7,AMTH3027-00,수치선형대수,3,김경수,3,NaN


취소 (학수번호-분반) : AMTH3029-00


,학수번호-분반,강좌명,대상학년,교수명,학점,특이사항
0,AMTH3003-00,미분기하학,3,한창용,3,NaN
2,AMTH3013-00,현대대수학I,3,김정산,3,NaN
7,AMTH3027-00,수치선형대수,3,김경수,3,NaN
